In [6]:
import rdflib
from rdflib.plugins.sparql import prepareQuery
from tabulate import tabulate

In [7]:
filename = "../../data/05/ABox.ttl"

In [8]:
text1 = '''CQ_5.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.
'''

query1 = '''
PREFIX : <http://purl.org/sirius/ontology/data/05/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <http://purl.org/sirius/ontology/hero/>

SELECT ?asset ?asset_part ?percentage
WHERE {
  ?asset a crm:E24_Physical_Human-Made_Thing ;
        crm:P46_is_composed_of ?asset_part .
  ?asset_part hero:representsAssetPercentage ?percentage .
}
'''

In [9]:
text2 = '''CQ_5.2
Return the assets that are part of the `house` asset and the contributing values assigned to them, along with their score, dimension, aspect, note, documentation, and time interval.
'''

query2 = '''
PREFIX : <http://purl.org/sirius/ontology/data/05/>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX hero: <http://purl.org/sirius/ontology/hero/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX tvc: <http://www.essepuntato.it/2012/04/tvc/>

SELECT DISTINCT ?asset_part ?value ?score ?dimension ?aspect ?note ?document ?time_interval_start ?time_interval_end
WHERE {
    ?asset a crm:E24_Physical_Human-Made_Thing ;
            crm:P46_is_composed_of ?asset_part .
    ?value_assessment hero:assignsValueTo ?asset_part ;
                        hero:assignsValue ?value .
    ?value hero:hasScore ?score ;
            hero:withDimension ?dimension ;
            hero:withinAspect ?aspect ;
            hero:hasNote ?note ;
            hero:isDocumentedBy ?document ;
            tvc:atTime ?time_interval .
    ?time_interval ti:hasIntervalStartDate ?time_interval_start ;
                    ti:hasIntervalEndDate ?time_interval_end .
}
'''

In [10]:
queries = [(text1, query1),
           (text2, query2)
           ]

g = rdflib.ConjunctiveGraph()
g.parse(filename, format="turtle", encoding="utf-8")

for query in queries:
    q = prepareQuery(query[1])
    results = g.query(q)
    print(query[0])
    table = []
    for row in results:
        table.append([row[var] for var in results.vars])
    print(tabulate(table, headers=results.vars, tablefmt="psql"))

CQ_5.1
Return the assets that are part of the `house` asset and the asset percentage they represent, in descending order.

+-----------------------------------------------+---------------------------------------------------+--------------+
| asset                                         | asset_part                                        |   percentage |
|-----------------------------------------------+---------------------------------------------------+--------------|
| http://purl.org/sirius/ontology/data/05/house | http://purl.org/sirius/ontology/data/05/building  |           50 |
| http://purl.org/sirius/ontology/data/05/house | http://purl.org/sirius/ontology/data/05/furniture |           20 |
| http://purl.org/sirius/ontology/data/05/house | http://purl.org/sirius/ontology/data/05/mosaics   |           20 |
| http://purl.org/sirius/ontology/data/05/house | http://purl.org/sirius/ontology/data/05/paintings |           10 |
+-----------------------------------------------+---------